In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
OUT_PATH: str = '../results/manifolds'

In [4]:
DATA_PATH: str = '../data/imdb'
DATASETS: list = [
    ('train', 'sample.train'),
    ('test', 'sample.test')
]

In [5]:
DIMS: list = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]
MODELS: list = [
    ('base', 'bert-base-uncased'),
    ('textattack', 'textattack/bert-base-uncased-imdb'),
    ('fabriceyhc', 'fabriceyhc/bert-base-uncased-imdb'),
    ('wakaka', 'Wakaka/bert-finetuned-imdb')
]

In [6]:
### Load Datasets into memory

In [7]:
data_config: dict = {
    'polarities': {
      "negative": 0,
      "positive": 1
    },
    'data_label': 'text',
    'target_label': 'sentiment'
}

In [8]:
from typing import Dict
from modules import Data

datasets: Dict[str, Data] = {
    label: Data(data_path=f"{DATA_PATH}.{name}.csv", **data_config)
    for label, name in DATASETS
}

In [9]:
for label, dataset in datasets.items():
    display(dataset.data)
    display(dataset.data['sentiment'].value_counts(normalize=True))

,text,sentiment
0,Aside from the horrendous acting and the ridic...,negative
1,Can such an ambient production have failed its...,positive
2,Oh-so-familiar comedy story about low-key nice...,negative
3,"In the beginning of this film, one of the comm...",positive
4,"Yes, some plots are a bit hard to follow, and ...",positive
...,...,...
1240,"I have watched 3 episodes of Caveman, and I ha...",negative
1241,"It's a very good movie, not only for the fans ...",positive
1242,"Modern, original, romantic story.Very good act...",positive
1243,The Straight Story is a multilevel exploration...,positive


positive    0.519679
negative    0.480321
Name: sentiment, dtype: float64

,text,sentiment
0,"Out of the top 24 lesbian films in my library,...",positive
1,I went to see this movie with a lady freind of...,positive
2,I have to say that I really liked UNDER SIEGE ...,negative
3,I paid attention and enjoyed the very rich exp...,positive
4,"A bad Quentin Tarantino rip off, at least I ho...",negative
...,...,...
1235,A group of obnoxious teens go to a former fune...,positive
1236,This is one of the movies having made signific...,positive
1237,I approached this movie with the understanding...,negative
1238,I watched fantabulosa! because over the last f...,positive


positive    0.504839
negative    0.495161
Name: sentiment, dtype: float64

In [10]:
### Load Encoder into Memory

In [11]:
encoder_config: dict = {
    'layers': [11]
}

In [12]:
from modules import Encoder

encoders: Dict[str, Encoder] = {
    label: Encoder({**{'model': ref}, **encoder_config})
    for label, ref in MODELS
}

In [13]:
### Compute manifolds and measure centroid distance and cluster dispersion

In [14]:
import numpy as np
import pandas as pd

In [15]:
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [16]:
from scipy.spatial import distance

def metric_computation(record: dict, groups: pd.core.groupby.GroupBy) -> None:

    # compute centroid means and dispersion for each cluster
    for label, group in groups:
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.sum(distance.pdist(group['reduced_embeds'].tolist()))

    record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item(0)

In [17]:
from typing import Generator
import torch

def reduce_analyse(data: pd.DataFrame, col: str, dim: list, default_dim: int = 768) -> Generator:

    embed_col: np.ndarray = torch.stack(data[col].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == default_dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        metric_computation(record, data.groupby(dataset.target_label))

        yield record

In [18]:
output_cols: list = ['dim', 'dispersion_positive', 'dispersion_negative', 'distance']

In [19]:
results: Dict[str, pd.DataFrame] = {}

In [20]:
for enc_label, encoder in encoders.items():
    for data_label, dataset in datasets.items():
        encoder.df_encode(dataset.data, col=dataset.data_label)
        results[f'{data_label}.{enc_label}'] = pd.DataFrame.from_records(
            list(reduce_analyse(
                dataset.data, encoder.col_name, DIMS,
                default_dim=encoder.dim)
            )
        )

In [21]:
for label, dataset in results.items():
    results[label][output_cols].to_csv(f'{OUT_PATH}/analysis.{label}.csv')
    display(results[label][output_cols])

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.846608e+06,1.474544e+06,1.191455
1,576,1.848007e+06,1.475781e+06,1.104581
2,384,1.847754e+06,1.475565e+06,1.115865
3,192,1.847403e+06,1.475146e+06,1.117858
4,96,1.845973e+06,1.473852e+06,1.167782
5,48,1.843396e+06,1.471407e+06,1.218857
6,24,1.838040e+06,1.465957e+06,1.328104
7,12,1.826143e+06,1.454249e+06,1.517147
8,6,1.812401e+06,1.435127e+06,1.248237
9,3,1.768678e+06,1.375959e+06,1.019506


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.682362e+06,1.571783e+06,1.254312
1,576,1.683568e+06,1.572977e+06,1.182686
2,384,1.683447e+06,1.572856e+06,1.180682
3,192,1.682942e+06,1.572307e+06,1.195615
4,96,1.681746e+06,1.571151e+06,1.225906
5,48,1.679057e+06,1.568448e+06,1.286972
6,24,1.674267e+06,1.563556e+06,1.354284
7,12,1.661568e+06,1.550616e+06,1.593289
8,6,1.634855e+06,1.520840e+06,1.905295
9,3,1.603535e+06,1.468301e+06,1.333726


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.136783e+06,1.716567e+06,7.983203
1,576,2.137147e+06,1.716806e+06,7.985439
2,384,2.136869e+06,1.716547e+06,7.986923
3,192,2.135925e+06,1.715649e+06,7.993095
4,96,2.133923e+06,1.713791e+06,8.005571
5,48,2.129540e+06,1.709696e+06,8.032312
6,24,2.120059e+06,1.700843e+06,8.090835
7,12,2.098074e+06,1.680481e+06,8.221065
8,6,2.042790e+06,1.630799e+06,8.522049
9,3,1.881023e+06,1.487288e+06,9.313700


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.997442e+06,1.865335e+06,7.293892
1,576,1.997749e+06,1.865593e+06,7.296332
2,384,1.997455e+06,1.865311e+06,7.298676
3,192,1.996618e+06,1.864474e+06,7.304216
4,96,1.994685e+06,1.862614e+06,7.317522
5,48,1.990453e+06,1.858556e+06,7.346356
6,24,1.981440e+06,1.849680e+06,7.408480
7,12,1.960951e+06,1.829719e+06,7.541790
8,6,1.907166e+06,1.777770e+06,7.878604
9,3,1.757716e+06,1.640257e+06,8.683845


,dim,dispersion_positive,dispersion_negative,distance
0,768,1.841314e+06,1.778661e+06,23.344327
1,576,1.841794e+06,1.779128e+06,23.344435
2,384,1.841676e+06,1.779056e+06,23.344594
3,192,1.841426e+06,1.778840e+06,23.344719
4,96,1.840792e+06,1.778376e+06,23.345179
5,48,1.839467e+06,1.777263e+06,23.346219
6,24,1.836317e+06,1.774793e+06,23.348690
7,12,1.828286e+06,1.768427e+06,23.355068
8,6,1.805838e+06,1.749138e+06,23.376483
9,3,1.707622e+06,1.662055e+06,23.559772


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.011053e+06,2.339915e+06,19.961293
1,576,2.011486e+06,2.340341e+06,19.961680
2,384,2.011368e+06,2.340235e+06,19.961977
3,192,2.011151e+06,2.340037e+06,19.961979
4,96,2.010568e+06,2.339567e+06,19.962522
5,48,2.009287e+06,2.338370e+06,19.963800
6,24,2.006147e+06,2.335904e+06,19.966276
7,12,1.998519e+06,2.329213e+06,19.974889
8,6,1.974660e+06,2.308466e+06,20.006246
9,3,1.879749e+06,2.233494e+06,20.204602


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.522510e+06,1.890411e+06,11.647890
1,576,2.522973e+06,1.890731e+06,11.650852
2,384,2.522782e+06,1.890579e+06,11.651119
3,192,2.522103e+06,1.889882e+06,11.653348
4,96,2.520431e+06,1.888266e+06,11.660106
5,48,2.516740e+06,1.884904e+06,11.674473
6,24,2.509036e+06,1.877356e+06,11.702982
7,12,2.489279e+06,1.859478e+06,11.784167
8,6,2.438433e+06,1.812340e+06,11.991117
9,3,2.300045e+06,1.676438e+06,12.581335


,dim,dispersion_positive,dispersion_negative,distance
0,768,2.327735e+06,2.037541e+06,11.718390
1,576,2.328194e+06,2.037943e+06,11.720677
2,384,2.327959e+06,2.037697e+06,11.721993
3,192,2.327250e+06,2.036991e+06,11.724361
4,96,2.325808e+06,2.035462e+06,11.729770
5,48,2.322370e+06,2.032033e+06,11.741664
6,24,2.314735e+06,2.024343e+06,11.771790
7,12,2.295898e+06,2.005609e+06,11.851487
8,6,2.244949e+06,1.957892e+06,12.057583
9,3,2.108218e+06,1.825098e+06,12.633208
